In [ ]:

# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import time
import sys
#
import io
import TNCmodem as TNC
from PIL import Image
import compression
#
from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones
from scipy import signal
from scipy import integrate
import threading

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
import bitarray
from  scipy.io.wavfile import read as wavread
import newax25 as ax25

import multiprocessing

from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign

In [ ]:
sd.query_devices()

In [ ]:
builtin_idx = 0
usb_idx = 2
sd.default.samplerate=48000
sd.default.channels = 1

In [ ]:
#create a queue to store packets
Qin = Queue.Queue()

def queueREC_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata
    Qin.put_nowait( indata.copy()[:,0] )  # Global queue

In [ ]:
sd.default.samplerate=48000
sd.default.channels = 1

st_rec = sd.Stream( device=(usb_idx, builtin_idx), callback=queueREC_callback)

In [ ]:
fs = 48000
modem = TNC.TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 1)

#start recording
st_rec.start()
#fname = ""

def decode_packets():
    print("decoding packets")
    starttime = time.time()
    npack = 0
    state = 0
    while(1):
        tmp = Qin.get() 

        packets  = modem.processBuffer(tmp)
        for pkt in packets: 
            ax = modem.decodeAX25(pkt)
            npack = npack + 1



            infostr = "%(n) 2i) | DEST: %(dest)s | SRC: %(src)s | DIGI: %(digi)s | %(info)s |" % {
                            'n': npack,
                            'dest': ax.destination,
                            'src': ax.source,
                            'digi': ax.digipeaters,
                            'info': ax.info.strip()
                        }
            print(infostr)

            if state == 0 and ax.destination[:5]=="BEGIN":
                f1 = open("rec_"+ax.info.decode(),"wb")
                #fname = "rec_"+ax.info.decode()
                #f1 = io.BufferedWriter(Bytes.IO())
                state = 1
            elif state == 1 and ax.destination[:3] == "END": 
                state = 2  
                break
            elif state == 1:
                f1.write(ax.info)
                print("write")
        if state == 2 :
            break

    print(time.time() - starttime)
    
    f1.close()

decode_packets()
st_rec.stop()
st_rec.close()

In [ ]:
# f1.seek(0)
#recv_img = Image.open(f1)
zip_recv = zipfile.ZipFile("rec_img.zip", 'r')
zip_recv.extractall(".")
zip_recv.close()
#inverse DCT function here
#new_img = ###FUNCTION(recv_img)
y = open("imagey.txt", 'r')
cr = open("imagecr.txt", 'r')
cb = open("imagecb.txt", 'r')
dec_image=decompress(y,cr,cb)
plt.imshow(dec_image)

rgbImage = imageio.imread("createrLake.tiff")
EE123_psnr(rgbImage,dec_image)